<a href="https://colab.research.google.com/github/spencerduke21/home_sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [14]:
# Import packages
from pyspark.sql import SparkSession
import urllib.request
import time
from pyspark.sql.functions import year, round

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [16]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [17]:
local_file_path = "/content/home_sales_revised.csv"
urllib.request.urlretrieve(url, local_file_path)

('/content/home_sales_revised.csv',
 <http.client.HTTPMessage at 0x7e512435ace0>)

In [19]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv(local_file_path, header=True, inferSchema=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [20]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

four_bedroom_df = df.filter(df['bedrooms'] == 4)
four_bedroom_df = four_bedroom_df.withColumn('Year', year('date'))
average_price_per_year = four_bedroom_df.groupBy('Year').avg('price').alias('Average_Price')
average_price_per_year = average_price_per_year.select('Year', round('avg(price)', 2).alias('Average_Price'))
average_price_per_year.show()

+----+-------------+
|Year|Average_Price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [21]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
three_bedroom_three_bathroom_df = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3))
average_price_per_year_built = three_bedroom_three_bathroom_df.groupBy('date_built').avg('price').alias('Average_Price')
average_price_per_year_built = average_price_per_year_built.select('date_built', round('avg(price)', 2).alias('Average_Price'))
average_price_per_year_built.show()

+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [22]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
filtered_df = df.filter((df['bedrooms'] == 3) &
                        (df['bathrooms'] == 3) &
                        (df['floors'] == 2) &
                        (df['sqft_living'] >= 2000))

average_price_per_year_built = filtered_df.groupBy('date_built').avg('price').alias('Average_Price')
average_price_per_year_built = average_price_per_year_built.select('date_built', round('avg(price)', 2).alias('Average_Price'))
average_price_per_year_built.show()

+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [27]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

start_time = time.time()

filtered_df = df.filter(df['price'] >= 350000)

filtered_df.createOrReplaceTempView("home_sales")

average_price_query = spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price FROM home_sales")
rounded_average_price = average_price_query.collect()[0]["Average_Price"]

view_rating = None
if rounded_average_price < 500000:
    view_rating = "Low"
elif rounded_average_price >= 500000 and rounded_average_price < 1000000:
    view_rating = "Moderate"
else:
    view_rating = "High"

end_time = time.time()

print("Average Price: $", rounded_average_price)
print("View Rating: ", view_rating)
print("Query Execution Time: ", end_time - start_time, " seconds")

Average Price: $ 473796.26
View Rating:  Low
Query Execution Time:  0.5322506427764893  seconds


In [28]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [29]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [31]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.


spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

filtered_df = df.filter(df['price'] >= 350000)


filtered_df.createOrReplaceTempView("home_sales")
spark.catalog.cacheTable("home_sales")


start_time_cached = time.time()
average_price_query = spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price FROM home_sales")
rounded_average_price = average_price_query.collect()[0]["Average_Price"]

view_rating = None
if rounded_average_price < 500000:
    view_rating = "Low"
elif rounded_average_price >= 500000 and rounded_average_price < 1000000:
    view_rating = "Moderate"
else:
    view_rating = "High"
end_time_cached = time.time()


spark.catalog.uncacheTable("home_sales")


start_time_uncached = time.time()
average_price_query_uncached = spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price FROM home_sales")
rounded_average_price_uncached = average_price_query_uncached.collect()[0]["Average_Price"]

view_rating_uncached = None
if rounded_average_price_uncached < 500000:
    view_rating_uncached = "Low"
elif rounded_average_price_uncached >= 500000 and rounded_average_price_uncached < 1000000:
    view_rating_uncached = "Moderate"
else:
    view_rating_uncached = "High"
end_time_uncached = time.time()


print("Cached Data:")
print("Average Price: $", rounded_average_price)
print("View Rating: ", view_rating)
print("Query Execution Time (Cached): ", end_time_cached - start_time_cached, " seconds")

print("\nUncached Data:")
print("Average Price: $", rounded_average_price_uncached)
print("View Rating: ", view_rating_uncached)
print("Query Execution Time (Uncached): ", end_time_uncached - start_time_uncached, " seconds")

Cached Data:
Average Price: $ 473796.26
View Rating:  Low
Query Execution Time (Cached):  0.2513606548309326  seconds

Uncached Data:
Average Price: $ 473796.26
View Rating:  Low
Query Execution Time (Uncached):  0.37146902084350586  seconds


In [32]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "/content/partitioned_home_sales_parquet"
filtered_df.write.partitionBy("date_built").parquet(output_path)

In [33]:
# 11. Read the parquet formatted data.
input_path = "/content/partitioned_home_sales_parquet"
partitioned_df = spark.read.parquet(input_path)
partitioned_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|e0726d4d-d595-407...|2022-01-08|424418|       3|        2|       2249|   13878|     2|         0|   4|      2017|
|131492a1-72e2-4a8...|2020-02-08|419199|       2|        3|       2062|    8876|     2|         0|   6|      2017|
|164f451a-413b-469...|2022-04-26|377485|       2|        2|       1937|    9033|     2|         0|  23|      2017|
|0ffe23e3-9d0f-442...|2021-11-18|429183|       3|        3|       2162|   12567|     1|         0|  49|      2017|
|141ef834-60cf-42e...|2021-10-13|399016|       2|        3|       1773|   13699|     1|         0|  23|      2017|
|fd8213d0-3872-4de...|2020-04-23|858900|       4|        5|       4465|    9873|

In [34]:
# 12. Create a temporary table for the parquet data.
input_path = "/content/partitioned_home_sales_parquet"
partitioned_df = spark.read.parquet(input_path)
partitioned_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|e0726d4d-d595-407...|2022-01-08|424418|       3|        2|       2249|   13878|     2|         0|   4|      2017|
|131492a1-72e2-4a8...|2020-02-08|419199|       2|        3|       2062|    8876|     2|         0|   6|      2017|
|164f451a-413b-469...|2022-04-26|377485|       2|        2|       1937|    9033|     2|         0|  23|      2017|
|0ffe23e3-9d0f-442...|2021-11-18|429183|       3|        3|       2162|   12567|     1|         0|  49|      2017|
|141ef834-60cf-42e...|2021-10-13|399016|       2|        3|       1773|   13699|     1|         0|  23|      2017|
|fd8213d0-3872-4de...|2020-04-23|858900|       4|        5|       4465|    9873|

In [35]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.


spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


partitioned_df.createOrReplaceTempView("partitioned_home_sales")


start_time_parquet = time.time()
average_price_query = spark.sql("SELECT ROUND(AVG(price), 2) AS Average_Price FROM partitioned_home_sales")
rounded_average_price = average_price_query.collect()[0]["Average_Price"]

view_rating = None
if rounded_average_price < 500000:
    view_rating = "Low"
elif rounded_average_price >= 500000 and rounded_average_price < 1000000:
    view_rating = "Moderate"
else:
    view_rating = "High"
end_time_parquet = time.time()


print("Parquet Data Query Results:")
print("Average Price: $", rounded_average_price)
print("View Rating: ", view_rating)
print("Query Execution Time (Parquet): ", end_time_parquet - start_time_parquet, " seconds")

Parquet Data Query Results:
Average Price: $ 473796.26
View Rating:  Low
Query Execution Time (Parquet):  0.3626573085784912  seconds


In [36]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [37]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
    print("The 'home_sales' table is still cached.")
else:
    print("The 'home_sales' table is not cached.")


The 'home_sales' table is not cached.
